In [6]:
import tkinter as tk
from tkinter import ttk, filedialog, messagebox
from PIL import Image, ImageTk
import cv2 as cv
import numpy as np

# Placeholder functions for image enhancements
def translate_image(img):
    rows, cols = img.shape[:2]
    M = np.float32([[1, 0, 100], [0, 1, 50]])
    return cv.warpAffine(img, M, (cols, rows))

def reflect_image(img):
    return cv.flip(img, 1)

def rotate_image(img):
    rows, cols = img.shape[:2]
    M = cv.getRotationMatrix2D((cols/2, rows/2), 30, 0.6)
    return cv.warpAffine(img, M, (cols, rows))

def crop_image(img):
    return img[50:250, 50:250]

def shear_image_x(img):
    rows, cols = img.shape[:2]
    M = np.float32([[1, 0.5, 0], [0, 1, 0]])
    return cv.warpAffine(img, M, (int(cols * 1.5), rows))

def shear_image_y(img):
    rows, cols = img.shape[:2]
    M = np.float32([[1, 0, 0], [0.5, 1, 0]])
    return cv.warpAffine(img, M, (cols, int(rows * 1.5)))

def blur_image(img):
    return cv.GaussianBlur(img, (7, 7), 0)

# Define the main application class
class ArtEnhancementApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Yemisi Shyllon Museum of Art - Image Enhancer")
        self.root.geometry("800x600")

        self.username = tk.StringVar()
        self.password = tk.StringVar()
        self.images = ["images/contemporary/bird_paradise.jpg", 
                       "images/contemporary/durbar.jpg", 
                       "images/contemporary/flexibility.jpg", 
                       "images/contemporary/milkmaids.jpg", 
                       "images/contemporary/widowhood.jpg",
                       "images/modern/dancer.jpg",
                       "images/modern/iya.jpg",
                       "images/modern/landscape_north.jpg",
                       "images/modern/nnamdi.jpg",
                       "images/modern/red_sky.jpg",
                       "images/traditional/benin_head.jpg",
                       "images/traditional/gelede_headdress.jpeg",
                       "images/traditional/igbo_ukwu_art.jpg",
                       "images/traditional/port_soldier.jpg",
                       "images/traditional/yorube_human_face.jpg"]

        self.create_login_screen()

    def create_login_screen(self):
        self.clear_screen()
        tk.Label(self.root, text="Username: admin & Password: admin").pack(pady=5)
        tk.Label(self.root, text="Username").pack(pady=5)
        tk.Entry(self.root, textvariable=self.username).pack(pady=5)
        tk.Label(self.root, text="Password").pack(pady=5)
        tk.Entry(self.root, textvariable=self.password, show="*").pack(pady=5)
        tk.Button(self.root, text="Login", command=self.check_login).pack(pady=20)

    def check_login(self):
        if self.username.get() == "admin" and self.password.get() == "admin":
            self.create_main_screen()
        else:
            messagebox.showerror("Error", "Incorrect Username or Password")

    def create_main_screen(self):
        self.clear_screen()

        canvas = tk.Canvas(self.root)
        scrollbar = tk.Scrollbar(self.root, orient="vertical", command=canvas.yview)
        scrollable_frame = ttk.Frame(canvas)

        scrollable_frame.bind(
            "<Configure>",
            lambda e: canvas.configure(
                scrollregion=canvas.bbox("all")
            )
        )

        canvas.create_window((0, 0), window=scrollable_frame, anchor="nw")
        canvas.configure(yscrollcommand=scrollbar.set)

        tk.Label(scrollable_frame, text="Select an Image to Enhance").grid(row=0, column=0, columnspan=4, pady=10)

        row, col = 1, 0
        for image_path in self.images:
            self.display_thumbnail(scrollable_frame, image_path, row, col)
            col += 1
            if col == 4:
                col = 0
                row += 1

        canvas.pack(side="left", fill="both", expand=True)
        scrollbar.pack(side="right", fill="y")

    def display_thumbnail(self, parent, image_path, row, col):
        image = Image.open(image_path)
        image.thumbnail((100, 100))
        tk_image = ImageTk.PhotoImage(image)
        btn = tk.Button(parent, image=tk_image, command=lambda path=image_path: self.open_image_editor(path))
        btn.image = tk_image  # Keep a reference to avoid garbage collection
        btn.grid(row=row, column=col, padx=5, pady=5)

    def open_image_editor(self, image_path):
        self.clear_screen()
        self.image = cv.imread(image_path)
        self.image_path = image_path

        self.display_image(self.image)

        tk.Button(self.root, text="Back", command=self.create_main_screen).pack(side="bottom", pady=10)

        tk.Label(self.root, text="Select Enhancement").pack(pady=10)
        enhancements = ["Translate", "Reflect", "Rotate", "Crop", "Shear X", "Shear Y", "Blur"]
        self.enhancement = tk.StringVar()
        self.enhancement.set(enhancements[0])
        tk.OptionMenu(self.root, self.enhancement, *enhancements).pack(pady=10)
        tk.Button(self.root, text="Apply Enhancement", command=self.apply_enhancement).pack(pady=10)
        tk.Button(self.root, text="Download Image", command=self.download_image).pack(pady=10)

    def display_image(self, image):
        if isinstance(image, np.ndarray):
            image = Image.fromarray(cv.cvtColor(image, cv.COLOR_BGR2RGB))
        tk_image = ImageTk.PhotoImage(image.resize((400, 400), Image.LANCZOS))
        panel = tk.Label(self.root, image=tk_image)
        panel.image = tk_image
        panel.pack()

    def apply_enhancement(self):
        enhancement = self.enhancement.get()
        if enhancement == "Translate":
            self.image = translate_image(self.image)
        elif enhancement == "Reflect":
            self.image = reflect_image(self.image)
        elif enhancement == "Rotate":
            self.image = rotate_image(self.image)
        elif enhancement == "Crop":
            self.image = crop_image(self.image)
        elif enhancement == "Shear X":
            self.image = shear_image_x(self.image)
        elif enhancement == "Shear Y":
            self.image = shear_image_y(self.image)
        elif enhancement == "Blur":
            self.image = blur_image(self.image)

        self.show_transformed_image(self.image)

    def show_transformed_image(self, img):
        new_window = tk.Toplevel(self.root)
        new_window.title("Transformed Image")
        new_window.geometry("400x400")

        if isinstance(img, np.ndarray):
            img = Image.fromarray(cv.cvtColor(img, cv.COLOR_BGR2RGB))
        tk_image = ImageTk.PhotoImage(img.resize((400, 400), Image.LANCZOS))
        panel = tk.Label(new_window, image=tk_image)
        panel.image = tk_image
        panel.pack()

    def download_image(self):
        save_path = filedialog.asksaveasfilename(defaultextension=".png", filetypes=[("PNG files", "*.png"), ("All files", "*.*")])
        if save_path:
            cv.imwrite(save_path, cv.cvtColor(self.image, cv.COLOR_RGB2BGR))
            messagebox.showinfo("Image Saved", "Your enhanced image has been saved successfully!")

    def clear_screen(self):
        for widget in self.root.winfo_children():
            widget.destroy()

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = ArtEnhancementApp(root)
    root.mainloop()
